In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D, LeakyReLU, Dropout, Reshape, Conv2DTranspose, BatchNormalization, GaussianDropout
from tensorflow.keras.activations import selu
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.datasets.cifar10 import load_data
from tensorflow.keras import Model
import numpy as np
from matplotlib import pyplot as plt
import os

from pylab import *
rcParams["figure.figsize"] = [8, 8]

In [0]:
(X, _), (_, _) = load_data()

In [0]:
inp = X.shape[-3:]
inp, inp[0], inp[1], inp[2]

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
path = 'drive/My Drive/DCGAN'

In [0]:
LABEL_SMOOTHING = 0
D_LR = .0002
D_NOISE = False
D_BN = False
G_BN = False
G_GAU_NOISE = False
GAN_LR = .0002
FLIP = False
FLIP_FREQ = 4
SELU = False

### Discriminator

In [0]:
def define_discriminator(in_shape = inp, BN = D_BN, selu = SELU, lr = D_LR, noise = D_NOISE):
    model = Sequential()
    if selu:
        if noise:
            model.add(GaussianDropout(.2, input_shape=in_shape))
            model.add(Conv2D(64, (3,3), padding='same', activation='selu'))
        else:
            model.add(Conv2D(64, (3,3), padding='same', activation='selu', input_shape=in_shape))
        #NOTE: Consider crippling the discriminator - make the number of filters much less than in the generator.
        # downsample
        model.add(Conv2D(128, (3,3), strides=(2,2), padding='same', activation='selu'))
        # downsample
        model.add(Conv2D(256, (3,3), strides=(2,2), padding='same', activation='selu'))
        # downsample
        model.add(Conv2D(256, (3,3), strides=(2,2), padding='same', activation='selu'))
        # classifier
        model.add(Flatten())
        model.add(Dropout(0.4))
        model.add(Dense(1, activation='sigmoid'))
        # compile model
        opt = Adam(lr = lr, beta_1 = 0.5)
        #opt = SGD(lr = lr, decay = 1e-6)
        model.compile(loss = tf.keras.losses.BinaryCrossentropy(label_smoothing = LABEL_SMOOTHING), 
                      optimizer = opt, metrics = [tf.keras.metrics.BinaryAccuracy()])
        return model

    else:
        if noise:
            model.add(GaussianDropout(.2, input_shape=in_shape))
            model.add(Conv2D(64, (3,3), padding='same'))
        else:
            model.add(Conv2D(64, (3,3), padding='same', input_shape=in_shape))
        #NOTE: Crippling the discriminator - make the number of filters much less than in the generator.
        model.add(LeakyReLU(alpha=0.2))
        # downsample
        model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
        if BN:
            model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))   
        # downsample
        model.add(Conv2D(256, (3,3), strides=(2,2), padding='same'))
        if BN:
            model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        # downsample
        model.add(Conv2D(256, (3,3), strides=(2,2), padding='same'))
        if BN:
            model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        # classifier
        model.add(Flatten())
        model.add(Dropout(0.4))
        model.add(Dense(1, activation='sigmoid'))
        # compile model
        opt = Adam(lr = lr, beta_1 = 0.5)
        #opt = SGD(lr = lr, decay = 1e-6)
        model.compile(loss = tf.keras.losses.BinaryCrossentropy(label_smoothing = LABEL_SMOOTHING), 
                      optimizer = opt, metrics = [tf.keras.metrics.BinaryAccuracy()])
        return model



# load and prepare cifar10 training images
def load_real_samples():
    # load cifar10 dataset
    (trainX, _), (_, _) = load_data()
    # convert from unsigned ints to floats
    X = trainX.astype('float32')
    # scale from [0,255] to [-1,1]
    X = (X - 127.5) / 127.5
    return X
 
# select real samples
def generate_real_samples(dataset, n_samples):
    # choose random instances
    ix = np.random.randint(0, dataset.shape[0], n_samples)
    # retrieve selected images
    X = dataset[ix]
    # generate 'real' class labels (1)
    y = np.ones((n_samples, 1))
    return X, y

# generate n fake samples with class labels
def fake_samples(n_samples):
    # generate uniform random numbers in [0,1]
    X = np.random.rand(inp[0] * inp[1] * inp[2] * n_samples)
    # update to have the range [-1, 1]
    X = -1 + X * 2
    # reshape into a batch of color images
    X = X.reshape((n_samples, inp[0], inp[1], inp[2]))
    # generate 'fake' class labels (0)
    y = np.zeros((n_samples, 1))
    return X, y
 
# train the discriminator model
def train_discriminator(model, dataset, n_iter=20, n_batch=128):
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_iter):
        # get randomly selected 'real' samples
        X_real, y_real = generate_real_samples(dataset, half_batch)
        # update discriminator on real samples
        _, real_acc = model.train_on_batch(X_real, y_real)
        # generate 'fake' examples
        X_fake, y_fake = fake_samples(half_batch)
        # update discriminator on fake samples
        _, fake_acc = model.train_on_batch(X_fake, y_fake)
        # summarize performance
        print('>%d real=%.0f%% fake=%.0f%%' % (i+1, real_acc*100, fake_acc*100))

In [0]:
# define the discriminator model
discriminator = define_discriminator()
discriminator.summary()
tf.keras.utils.plot_model(discriminator, to_file=os.path.join(path, 'discriminator_plot.png'), show_shapes=True, show_layer_names=True)

In [0]:
# load image data
dataset = load_real_samples()
# fit the model
train_discriminator(discriminator, dataset)

### Generator

In [0]:
# define the standalone generator model
def define_generator(latent_dim, BN = G_BN, selu = SELU, GauNoise = G_GAU_NOISE):
    model = Sequential()
    if selu:
        # foundation for 4x4 image
        n_nodes = 256 * 4 * 4
        model.add(Dense(n_nodes, input_dim=latent_dim, activation='selu'))
        if GauNoise:
            model.add(GaussianDropout(.2)) # gaussian noise
        model.add(Reshape((4, 4, 256)))
        model.add(Dropout(0.5))
        # upsample to 8x8
        model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', activation='selu'))
        if GauNoise:
            model.add(GaussianDropout(.2))
        model.add(Dropout(0.5))
        # upsample to 16x16
        model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', activation='selu'))
        if GauNoise:
            model.add(GaussianDropout(.2))
        model.add(Dropout(0.5))
        # upsample to 32x32
        model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', activation='selu'))
        if GauNoise:
            model.add(GaussianDropout(.2))
        model.add(Dropout(0.5))
        # output layer
        model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
        return model

    else:

        # foundation for 4x4 image
        n_nodes = 256 * 4 * 4
        model.add(Dense(n_nodes, input_dim=latent_dim))
        if GauNoise:
            model.add(GaussianDropout(.2)) # gaussian noise 
        if BN:
            model.add(BatchNormalization())
        model.add(Reshape((4, 4, 256)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        # upsample to 8x8
        model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
        if GauNoise:
            model.add(GaussianDropout(.2))
        if BN:
            model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        # upsample to 16x16
        model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
        if GauNoise:
            model.add(GaussianDropout(.2))
        if BN:
            model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        # upsample to 32x32
        model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
        if GauNoise:
            model.add(GaussianDropout(.2)) 
        if BN:
            model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.5))
        # output layer
        model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
        return model

 
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space 
    # calling the randn() NumPy function for generating arrays of random numbers drawn from a standard Gaussian
    x_input = np.random.randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input
 
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    X = g_model.predict(x_input)
    # create 'fake' class labels (0)
    y = np.zeros((n_samples, 1))
    return X, y

In [0]:
# size of the latent space
latent_dim = 100
# define the discriminator model
model = define_generator(latent_dim)
# generate samples
n_samples = 49
X, _ = generate_fake_samples(model, latent_dim, n_samples)
# scale pixel values from [-1,1] to [0,1]
X = (X + 1) / 2.0
# plot the generated samples
for i in range(n_samples):
    # define subplot
    plt.subplot(7, 7, 1 + i)
    # turn off axis labels
    plt.axis('off')
    # plot single image
    plt.imshow(X[i])
# show the figure
plt.show()

Note: the generator model is not compiled and does not specify a loss function or optimization algorithm. This is because the generator is not trained directly. We first update the discriminator model with real and fake samples, then update the generator via the composite model (stack). That way the generator is forced to produce more realistic images.

In [0]:
# define the generator model
generator = define_generator(latent_dim)
generator.summary()
tf.keras.utils.plot_model(generator, to_file=os.path.join(path, 'generator_plot.png'), show_shapes=True, show_layer_names=True)

## Stack models

In [0]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model, lr = GAN_LR):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(g_model)
    # add the discriminator
    model.add(d_model)
    # compile model
    opt = Adam(lr = lr, beta_1 = 0.5)
    model.compile(loss = tf.keras.losses.BinaryCrossentropy(label_smoothing = LABEL_SMOOTHING), optimizer = opt)
    return model

In [0]:
# create and save a plot of generated images
def save_plot(examples, epoch, n=7):
    # scale from [-1,1] to [0,1]
    examples = (examples + 1) / 2.0
    # plot images
    for i in range(n * n):
      # define subplot
      plt.subplot(n, n, 1 + i)
      # turn off axis
      plt.axis('off')
      # plot raw pixel data
      plt.imshow(examples[i])
    # save plot to file
    if (epoch+1) % 10 == 0: 
        filename = os.path.join(path, 'generated_plot_e%03d.png' % (epoch+1))
        plt.savefig(filename)
        plt.show()
    plt.close()
 
# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=150):
    # prepare real samples
    X_real, y_real = generate_real_samples(dataset, n_samples)
    # evaluate discriminator on real examples
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    # prepare fake examples
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    # evaluate discriminator on fake examples
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    # summarize discriminator performance
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
    # save plot
    save_plot(x_fake, epoch)

def flip_labels (y, pct, n):
    # randomly flip labels from 0 to 1 and 1 to 0 for discriminator. Adds noise
    idx = np.random.choice(np.arange(y.size), replace=False, size=int(y.size * pct))
    y[idx] = n

    return y
 
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=200, n_batch=128):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_epochs):
        # enumerate batches over the training set
        for j in range(bat_per_epo):
            # get randomly selected 'real' samples
            X_real, y_real = generate_real_samples(dataset, half_batch)
            # flip some of the labels to 0
            #y_real = flip_labels(y_real, .1, 0)
            # NOTE: Label flipping - train every third batch on flipped labels
            if (i+1) % FLIP_FREQ == 0 and FLIP:
                y_real = np.full_like(y_real, 0)
            # update discriminator model weights
            d_loss1, _ = d_model.train_on_batch(X_real, y_real)
            # generate 'fake' examples
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # flip some of the labels to 1
            #y_fake = flip_labels(y_fake, .1, 0)
            if (i+1) % FLIP_FREQ == 0 and FLIP:
                y_fake = np.full_like(y_fake, 1)
            # update discriminator model weights
            d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
            # prepare points in latent space as input for the generator
            X_gan = generate_latent_points(latent_dim, n_batch)
            # create inverted labels for the fake samples
            y_gan = np.ones((n_batch, 1))
            # update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            # summarize loss on this batch
            if (j+1) % bat_per_epo == 0:
                print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' % (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
      # evaluate the model performance, sometimes
        if (i+1) % 1 == 0:
            summarize_performance(i, g_model, d_model, dataset, latent_dim)
        if (i+1) % 20 == 0:
            # save the generator model file
            filename = os.path.join(path, 'generator_model_%03d.h5') % (i+1)
            g_model.save(filename)
 
# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# summarize gan model
gan_model.summary()
# plot gan model
tf.keras.utils.plot_model(gan_model, to_file = os.path.join(path, 'gan_plot.png'), show_shapes=True, show_layer_names=True)

In [0]:
# dummy test the stacked model
def train_gan(gan_model, latent_dim = 100, n_iter=20, n_batch=128):
    for i in range(n_iter):
        # get randomly selected 'real' samples
        X_gan = generate_latent_points(latent_dim, n_batch)
        # create inverted labels for the fake samples
        y_gan = np.ones((n_batch, 1))
        # update the generator via the discriminator's error
        g_loss = gan_model.train_on_batch(X_gan, y_gan)
        # summarize loss on this batch
        print('>%d, g=%.3f' % (i+1, g_loss))

train_gan(gan_model)

## Execution

In [0]:
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)